# Artificial Intelligence for Games
- [Udemy Course](https://derivco.udemy.com/course/artificial-intelligence-for-simple-games/)

### Environment Setup

In [197]:
%run ../../../utilities/reinforcement_learning/environments/crypto_pair_trading_environment.ipynb

In [198]:
import pickle

In [199]:
model_file_path = '../../market_trading/data/featurized_market_data.p' #'./data/featurized_market_data.p' 

with open(model_file_path, 'rb') as fp:
    featurized_market_data = pickle.load(fp)

In [200]:
pair_name = 'AAVE_BTC'
pair_data = featurized_market_data[pair_name]

# This dataset contains open, close, high, low, volume and any numerical features / indicators you like.
price_movement_df = pair_data

In [201]:
env = CryptoPairTradingEnv(pair_data, pair_name)

### Genetic Algorithms

#### Imports

In [202]:
import numpy as np

#### Configuration

#### Utilities

In [203]:
class DNA():
    def __init__(self, dna, dna_length, default_gene=0):
        self.dna = dna.copy()
        self.dna_length = dna_length
        self.default_gene = default_gene
        
        while len(self.dna) < dna_length:
            self.dna.append(default_gene)
            
    def crossover(self, dna2, min_gene_value, max_gene_value, mutation_rate=0.1):
        new_dna_sequence = []
        
        # Simple cross-over function based on a 50% probability to take either parent's genes.
        for gene_index in range(self.dna_length):
            if np.random.rand() <= 0.5:
                new_dna_sequence.append(self.dna[gene_index])
            else:
                new_dna_sequence.append(dna2.dna[gene_index])
                
        # Mutation
        for gene_index in range(self.dna_length):
            if np.random.rand() <= mutation_rate:
                new_dna_sequence[gene_index] = np.random.randint(low=min_gene_value, high=(max_gene_value + 1))
        
        return DNA(new_dna_sequence, self.dna_length, self.default_gene)

In [204]:
class EnvironmentActionOptimizer():
    def __init__(self, env, population_size=100):
        self.population_size = population_size
        self.env = env
        self.reset()
        
    def generate_population(self, population_size):
        print(f'Generating a population size of {population_size}.')
        
        population = []
        
        for index in range(population_size):
            state = env.reset()
            done = False
            actions = []
            
            while not done:
                action = env.action_space.sample()
                state, reward, done, info = env.step(action)
                actions.append(action)
                
            population.append(DNA(actions, env.data.shape[0] - 1))
            
        return population

    def reset(self):
        self.population = self.generate_population(self.population_size)

In [205]:
optimizer = EnvironmentActionOptimizer(env)

Generating a population size of 100.


C:\Users\divanma\.conda\envs\windows-ml-station\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [206]:
crossed_over_dna = optimizer.population[0].crossover(optimizer.population[1], min_gene_value=min([key.value for key in Actions]), max_gene_value=max([key.value for key in Actions]))